# Hoja de Trabajo 2

- Josue Marroquin 22397
- Sebastian Huertas 22295

In [4]:
# Libs
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

## Ejercicio 1 - Experimentación Práctica

### Task 1 - Preparación del conjunto de datos
Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos
en conjuntos de entrenamiento y validación.

In [5]:
# Cargar el conjunto de datos de Iris
iris = load_iris()
X = iris.data  # Características: sepal length, sepal width, petal length, petal width
y = iris.target  # Etiquetas: 0=setosa, 1=versicolor, 2=virginica

# Crear DataFrame para mejor visualización
feature_names = iris.feature_names
target_names = iris.target_names

df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df['species'] = df['target'].map({0: target_names[0], 1: target_names[1], 2: target_names[2]})

print("Información del conjunto de datos:")
print(f"Forma del dataset: {X.shape}")
print(f"Número de características: {X.shape[1]}")
print(f"Número de muestras: {X.shape[0]}")
print(f"Clases: {target_names}")
print(f"Distribución de clases: {np.bincount(y)}")

# Mostrar las primeras filas
print("\nPrimeras 5 filas del dataset:")
print(df.head())

# Dividir el conjunto de datos en entrenamiento y validación (80% - 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Mantener la proporción de clases en ambos conjuntos
)

print(f"\nConjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Conjunto de validación: {X_val.shape[0]} muestras")
print(f"Distribución en entrenamiento: {np.bincount(y_train)}")
print(f"Distribución en validación: {np.bincount(y_val)}")

# Estandarizar las características (opcional pero recomendado para muchos algoritmos)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print("\nDatos preparados exitosamente para entrenamiento y validación")
print("Variables disponibles:")
print("- X_train, X_val: características originales")
print("- X_train_scaled, X_val_scaled: características estandarizadas")
print("- y_train, y_val: etiquetas")
print("- df: DataFrame completo con información descriptiva")

Información del conjunto de datos:
Forma del dataset: (150, 4)
Número de características: 4
Número de muestras: 150
Clases: ['setosa' 'versicolor' 'virginica']
Distribución de clases: [50 50 50]

Primeras 5 filas del dataset:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target species  
0       0  setosa  
1       0  setosa  
2       0  setosa  
3       0  setosa  
4       0  setosa  

Conjunto de entrenamiento: 120 muestras
Conjunto de validación: 30 muestras
Distribución en entrenamiento: [40 40 40]
Distribución en validación: [10 10 10]

Datos preparados exi